<img src="https://www.dept.aueb.gr/sites/default/files/BA.jpg" title="PROGRAME Logo"/>

### <span style="color:darkblue"> <hr>Athens University of Economics and Business<br> M.Sc. Business Analytics (Part Time)<br>Mining Big Datasets, Kotidis Ioannis / Ioanna Fillipidou <hr><br>Athanasios Alexandris p2822202<br> Ioannis Demertzis p2822210<br> 28 – 05 – 23<hr>
</span>


## `Assignment I`
This notebook contains the task descriptions for the first assignment, as well as the fully commented code produced to deliver these tasks along with comments and analysis.

### <hr>`Task 1- Import and pre-process the dataset with users`

Download the movieLens dataset from moodle. This dataset includes **100.000** ratings (1-5) from **943** users on **1682** movies. Each user has rated **at least 20 movies**. There are **3 files** for the dataset:  
- the **users.txt** file contains id, age, gender, occupation, and postcode separated by | 
- the **movies.txt** file contains id, title (with release year) and some other information not related with the assignment separated by | 
- the **ratings.txt** file (tab separated) which contains userid, movieid, rating (1-5) and timestamp. 

For this assignment you will only use the set of movies that **a user has rated and not the ratings**. 
In your report you should **describe in detail** any processing and conversion you made to the original data and the **reasons** it was necessary.<hr>

#### Handling the Import Error
While trying to import the files an error came up regarding the file's encoding. \
To tackle this, we first tried to identify the encoding of each file, as seen in the following code chunk, and then decided which encoding was the correct one for reading the files.

In [1]:
# Import required libraries
import pandas as pd
import chardet

# List of file names
files = ['users.txt', 'movies.txt', 'ratings.txt']

# Iterate over file names
for filename in files:
    
    # Detect the encoding of a text file
    
    # "With" statement to ensure that the file is properly closed after it is read.
    # "Open" function to open the file in binary mode "rb" and store to the variable "f"
    with open(filename, 'rb') as f:
        
        # "Read" function and "detect" function to identify the encoding
        result = chardet.detect(f.read()) 
    
    # Print the detected encoding
    print(f'{filename}: {result["encoding"]}')

users.txt: ascii
movies.txt: ISO-8859-1
ratings.txt: ascii


In [2]:
# Read the data files by specifying the correct encoding
users = pd.read_csv('users.txt', 
                    sep='|', 
                    names=['userid', 'age', 'gender', 'occupation', 'postcode'], 
                    encoding='ascii')

movies = pd.read_csv('movies.txt',
                     usecols=[0, 1],
                     sep='|', 
                     names=['movieid', 'title'], 
                     encoding='ISO-8859-1')

ratings = pd.read_csv('ratings.txt', 
                      sep='\t', 
                      names=['userid', 'movieid', 'rating', 'timestamp'], 
                      encoding='ascii')

In [3]:
# Merge the dataframes
data = pd.merge(pd.merge(users, ratings, on='userid'), movies, on='movieid')

In [4]:
# Drop unnecessary columns
data = data.drop(['rating', 'timestamp'], axis=1)

#### Explaining the Steps

1- To combine the three datataframes into one complete dataset, we used the **`merge()`** function. An inner join was performed to only include rows where there was a match between the **userid** and **movieid** columns in the users, movies, and ratings dataframes. This means that the resulting dataframe contained information about users who had rated movies and movies that had been rated by users.

2- The **rating** and **timestamp** columns were dropped as it was requested to form a dataset using only the set of movies that a user has rated and not the ratings themselves.

In [5]:
# Display resulting dataframe
data

,userid,age,gender,occupation,postcode,movieid,title
0,1,24,M,technician,85711,61,Three Colors: White (1994)
1,13,47,M,educator,29206,61,Three Colors: White (1994)
2,18,35,F,other,37212,61,Three Colors: White (1994)
3,58,27,M,programmer,52246,61,Three Colors: White (1994)
4,59,49,M,educator,08403,61,Three Colors: White (1994)
...,...,...,...,...,...,...,...
99995,863,17,M,student,60089,1679,B. Monkey (1998)
99996,863,17,M,student,60089,1678,Mat' i syn (1997)
99997,863,17,M,student,60089,1680,Sliding Doors (1998)
99998,896,28,M,writer,91505,1681,You So Crazy (1994)


#### <hr>Data Cleaning<hr>

For data cleaning purposes, we checked for missing values in the data dataframe by using the **`isnull()`** function. No missing values in any of the variables were found.

Also, in order to gain a better understanding of the data and identify any "extreme" or "strange" values, we used the **`describe()`** function on the data dataframe. This provided us with descriptive statistics, including count, mean, standard deviation, minimum, quartiles, and maximum values for each numerical column in the dataframe.

Nothing unusual was detected.

In [6]:
# Check for missing values
data.isnull().sum()

userid        0
age           0
gender        0
occupation    0
postcode      0
movieid       0
title         0
dtype: int64

In [7]:
# Understand if there are "extreme" or "strange" values (eg. age 1000)
data.describe()

,userid,age,movieid
count,100000.00000,100000.000000,100000.000000
mean,462.48475,32.969850,425.530130
std,266.61442,11.562623,330.798356
min,1.00000,7.000000,1.000000
25%,254.00000,24.000000,175.000000
50%,447.00000,30.000000,322.000000
75%,682.00000,40.000000,631.000000
max,943.00000,73.000000,1682.000000


In [8]:
# Get unique values for "postcode"
unique_postcodes= data['postcode'].unique()

# Sort unique values in lexicographic order
unique_postcodes.sort()

# Display the head and tail of the unique post codes
postcodes = pd.DataFrame(unique_postcodes, columns=['postcode'])
postcodes

,postcode
0,00000
1,01002
2,01040
3,01080
4,01331
...,...
790,V0R2M
791,V1G4L
792,V3N4P
793,V5A2B


Based on the above it seemed tha most postal codes contained 5 numbers but there were also 17 postcodes which were a mix of letters and numbers. 

After a quick search online, these 2 different post code types corresponded to USA and Canada postcodes.

In [9]:
# Get unique values for "οccupation"
unique_occupation= data['occupation'].unique()

# Sort unique values in lexicographic order
unique_occupation.sort()

# Print sorted unique values
print(unique_occupation)

['administrator' 'artist' 'doctor' 'educator' 'engineer' 'entertainment'
 'executive' 'healthcare' 'homemaker' 'lawyer' 'librarian' 'marketing'
 'none' 'other' 'programmer' 'retired' 'salesman' 'scientist' 'student'
 'technician' 'writer']


### <hr>`Task 2 - Compute exact Jaccard similarity of users`

To assess the similarity between users you should compute the **exact Jaccard Similarity for all pairs of users** and only output the pairs of users (unique) that have similarity **at least 0,5 (>=50%)**. For each pair denote their ids and the similarity score.<hr>

#### The Process of Calculating the Similarity
The Jaccard Similarity between two users is defined as the size of the intersection of the sets of movies they have rated divided by the size of the union of the sets of movies they have rated.

Brute force calculating the Jaccard similarity can be computationally expensive for large datasets because it computes it for all pairs of users, which has a time complexity of `O(n^2)` where `n` is the number of users.

For that reason a dictionary was used to store the sets of movies rated by each user. The dictionary was populated by iterating over the unique user ids and indexing the dataframe using boolean indexing to get the set of movies rated by each user and then calculating the Jaccard similarity between them.

**Process explanation**: 
- First, we obtained the unique user IDs from the dataset. 
- Then, we initialized a dictionary called **movies_user** to store sets of movies rated by each user. To populate this dictionary, we iterated over the unique user IDs. For each user ID, we selected the rows where the **userid** value matched the current user ID. From these selected rows, we extracted the **movieid** column and converted it into a set using the set function.
- Finally, we assigned this set of movies to the corresponding key in the **movies_user** dictionary. This way, we obtained a dictionary that mapped each user ID to a set of movies they had rated.

In [10]:
# Get the unique user ids
userids = data['userid'].unique()

# Initialize a dictionary to store sets of movies rated by each user
movies_user = {}

# Populate dictionary with sets of movies rated by each user

# Iterate over the unique user ids and select only the rows where the "userid" value is equal to the current user id. 
for userid in userids:
    
    # The "movieid" column is then selected and converted into a set using the "set" function.
    # This set is then assigned to the corresponding key in the movies dictionary
    movies_user[userid] = set(data.loc[data['userid'] == userid, 'movieid'])

To perform pairwise iterations between users, we imported the **`combinations`** module from the itertools library. After initializing an empty list called **results**, we iterated over all pairs of users using the **`combinations()`** function. For each pair of users, we accessed their respective sets of movies rated, **movies_1** and **movies_2**, from the **movies_user** dictionary.

To compute the Jaccard similarity between the two sets, we calculated the intersection of movies rated by both users and the union of movies rated by either user. The Jaccard similarity was then determined by dividing the size of the intersection by the size of the union. 
If the Jaccard similarity was found to be at least 50%, we stored the result by appending a tuple containing the user IDs and the Jaccard similarity to the results list.

By applying this process to all possible pairs of users, we obtained a list of results representing pairs of users with a Jaccard similarity of at least 50%.

In [11]:
# Import required library to perform pairwise iterations
from itertools import combinations

# Calculate the start time of the code execution
import time
start_time = time.time()

# Initialize a list to store the results
results = []

# Iterate over all pairs of users
for user1, user2 in combinations(userids, 2):
    
    # Get sets of movies rated by each user
    
    # Set of movies rated by the first user from the movies dictionary
    movies_1 = movies_user[user1]
    
    # Set of movies rated by the second user from the movies dictionary
    movies_2 = movies_user[user2]
    
    # Compute the Jaccard similarity
    
    # Intersection = only the movies that are present in both sets
    intersection = movies_1 & movies_2
    
    # Union = only the movies that are present in either set
    union = movies_1 | movies_2
    
    # Jaccard Similarity = the size of the intersection divided by the size of the union
    jaccard_similarity = len(intersection) / len(union)
    
    # Check if Jaccard similarity is at least 50%
    if jaccard_similarity >= 0.5:
        
        # Store result
        results.append((user1, user2, jaccard_similarity))

# Calculate the end time and elapsed time of the code execution
end_time = time.time()
elapsed_time = end_time - start_time

# Print the elapsed time of the code execution
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 5.247977256774902 seconds


Next, we displayed the pair of users that have similarity greater than 0.5, sorted in descending order.

In [12]:
# Create result dataframe
result_df = pd.DataFrame(results, columns=['user1', 'user2', 'similarity'])

# Display result dataframe
result_df.sort_values(by='similarity', ascending=False)

,user1,user2,similarity
9,408,898,0.838710
4,328,788,0.672956
8,489,587,0.629921
2,826,600,0.545455
7,451,489,0.533333
5,674,879,0.521739
3,554,764,0.517007
0,197,826,0.512987
1,197,600,0.500000
6,879,800,0.500000


Then, we found the pair with the maximum Jaccard similarity from the results list. The pair with the highest similarity was determined by using the **`max()`** function, with the key parameter set to a **`lambda`** function that accessed the Jaccard similarity value from index position 2. The user IDs of this pair were assigned to **user1** and **user2**.

To get the sets of movies rated by each user, we accessed the respective user's set of movies from the **movies_user** dictionary. The set of movies rated by user1 was assigned to **movies_1**, and the set of movies rated by user2 was assigned to **movies_2**.

In [13]:
# Find the pair with maximum Jaccard similarity (index 2)
max_pair = max(results, key=lambda x: x[2])
user1, user2 = max_pair[0], max_pair[1]

# Get sets of movies rated by each user   
# Set of movies rated by the first user from the movies dictionary
movies_1 = movies_user[user1]
# Set of movies rated by the second user from the movies dictionary
movies_2 = movies_user[user2]

We then obtained the movie names corresponding to the movie IDs in **movies_1** and **movies_2**. This was achieved by creating a dictionary (**movie_names**) that mapped movie IDs to movie names from the movies dataset. We used this dictionary to retrieve the movie names for each user by iterating over their respective sets of movies.

Next, we calculated the intersection of the two sets (**movies_1** and **movies_2**) to identify the common movies rated by both users. We obtained the movie names of this intersection by referencing the **movie_names** dictionary.

Similarly, we calculated the union of the two sets to determine the total movies seen by the two users combined. We retrieved the movie names of this union using the **movie_names** dictionary.

Finally, we printed the results in a sorted manner. We displayed the movies seen by user1 along with the count of movies. Similarly, we displayed the movies seen by user2 and their count. We also printed the movies seen by both users and their count. Additionally, we displayed the total movies seen by both users and their count.

In [14]:
# Get all the movies names per movie id
movie_names = dict(zip(movies['movieid'], movies['title']))

In [15]:
# Get the movie names for user 1 from the pair with highest similarity
user1_movies = [movie_names[movie_id] for movie_id in movies_1]
print(f'Movies Seen From User 1:\n{sorted(user1_movies)}')
print(f'Number of Movies:\n{len(user1_movies)}')

Movies Seen From User 1:
['Air Force One (1997)', 'Apt Pupil (1998)', 'Conspiracy Theory (1997)', 'Contact (1997)', 'Cop Land (1997)', 'English Patient, The (1996)', 'Everyone Says I Love You (1996)', 'Gattaca (1997)', 'Good Will Hunting (1997)', 'Indian Summer (1996)', 'Jackal, The (1997)', 'Kolya (1996)', 'L.A. Confidential (1997)', 'Liar Liar (1997)', 'Lost Highway (1997)', 'Midnight in the Garden of Good and Evil (1997)', 'Mouse Hunt (1997)', 'Rainmaker, The (1997)', 'Rocket Man (1997)', 'Saint, The (1997)', 'Scream (1996)', 'Spawn (1997)', 'Starship Troopers (1997)', 'Titanic (1997)', 'Tomorrow Never Dies (1997)', 'U Turn (1997)', 'Wag the Dog (1997)']
Number of Movies:
27


In [16]:
# Get the movie names for user 2 from the pair with highest similarity
user2_movies = [movie_names[movie_id] for movie_id in movies_2]
print(f'Movies Seen From User 2:\n{sorted(user2_movies)}')
print(f'Number of Movies:\n{len(user2_movies)}')

Movies Seen From User 2:
['Air Force One (1997)', 'Alien: Resurrection (1997)', 'Apt Pupil (1998)', 'As Good As It Gets (1997)', 'Conspiracy Theory (1997)', 'Contact (1997)', 'Cop Land (1997)', 'Deceiver (1997)', 'English Patient, The (1996)', 'Everyone Says I Love You (1996)', 'Gattaca (1997)', 'Good Will Hunting (1997)', 'Indian Summer (1996)', 'Jackal, The (1997)', 'Jungle2Jungle (1997)', 'Kolya (1996)', 'L.A. Confidential (1997)', 'Lost Highway (1997)', 'Midnight in the Garden of Good and Evil (1997)', 'Mouse Hunt (1997)', 'Rainmaker, The (1997)', 'Rocket Man (1997)', 'Saint, The (1997)', 'Scream (1996)', 'Spawn (1997)', 'Starship Troopers (1997)', 'Titanic (1997)', 'Tomorrow Never Dies (1997)', 'U Turn (1997)', 'Wag the Dog (1997)']
Number of Movies:
30


In [17]:
# Compute the intersection of the two sets
intersection = movies_1 & movies_2

# Get movie names of the intersection
intersection_movie_names = [movie_names[movie_id] for movie_id in intersection]

# Print output sorted in ascending order
print(f'Movies Seen From Both Users:\n{sorted(intersection_movie_names)}')
print(f'Number of Common Movies:\n{len(intersection_movie_names)}')

Movies Seen From Both Users:
['Air Force One (1997)', 'Apt Pupil (1998)', 'Conspiracy Theory (1997)', 'Contact (1997)', 'Cop Land (1997)', 'English Patient, The (1996)', 'Everyone Says I Love You (1996)', 'Gattaca (1997)', 'Good Will Hunting (1997)', 'Indian Summer (1996)', 'Jackal, The (1997)', 'Kolya (1996)', 'L.A. Confidential (1997)', 'Lost Highway (1997)', 'Midnight in the Garden of Good and Evil (1997)', 'Mouse Hunt (1997)', 'Rainmaker, The (1997)', 'Rocket Man (1997)', 'Saint, The (1997)', 'Scream (1996)', 'Spawn (1997)', 'Starship Troopers (1997)', 'Titanic (1997)', 'Tomorrow Never Dies (1997)', 'U Turn (1997)', 'Wag the Dog (1997)']
Number of Common Movies:
26


In [18]:
# Compute the union of the two sets
union = movies_1 | movies_2

# Get movie names of the union
union_movie_names = [movie_names[movie_id] for movie_id in union]

# Print output sorted in ascending order
print(f'Total Movies Seen From The Users:\n{sorted(union_movie_names)}')
print(f'Number of Total User Movies:\n{len(union_movie_names)}')

Total Movies Seen From The Users:
['Air Force One (1997)', 'Alien: Resurrection (1997)', 'Apt Pupil (1998)', 'As Good As It Gets (1997)', 'Conspiracy Theory (1997)', 'Contact (1997)', 'Cop Land (1997)', 'Deceiver (1997)', 'English Patient, The (1996)', 'Everyone Says I Love You (1996)', 'Gattaca (1997)', 'Good Will Hunting (1997)', 'Indian Summer (1996)', 'Jackal, The (1997)', 'Jungle2Jungle (1997)', 'Kolya (1996)', 'L.A. Confidential (1997)', 'Liar Liar (1997)', 'Lost Highway (1997)', 'Midnight in the Garden of Good and Evil (1997)', 'Mouse Hunt (1997)', 'Rainmaker, The (1997)', 'Rocket Man (1997)', 'Saint, The (1997)', 'Scream (1996)', 'Spawn (1997)', 'Starship Troopers (1997)', 'Titanic (1997)', 'Tomorrow Never Dies (1997)', 'U Turn (1997)', 'Wag the Dog (1997)']
Number of Total User Movies:
31


### <hr>`Task 3 - Compute similarity using Min-hash signatures`

In this step you compute min-hash signatures for each user and use them to evaluate their similarity.

**Description of hash functions**: use the following family of hash functions: `ha,b(x)=(ax+b) mod R`, with a,b random integers in the interval (0,R) and R a large enough prime number that you may want to finetune in your initial experimentation. Make sure that each hash function uses different values of a,b pairs.

**Evaluation of Min-hashing**: Use 50, 100, and 200 hash functions. For each value, output the pair of users that have estimated similarity at least 0.5, and report the number of false positives and false negatives (against the exact Jaccard similarity) that you obtain. For the false positives and negatives, report the averages for 5 different runs using different functions. Comment on how the number of hash functions affects the false positive and negatives figures.<hr>

#### The Process of Computing Similarity Using Min-hash Signatures

To begin with, we defined the following two following functions: 

- **`get_similarity()`**: 
This function calculates the similarity between two signatures. It first counts the number of elements that are equal in both signatures by using a generator expression and the **`zip()`** function to compare corresponding elements of the two signatures. Then, it calculates the **similarity** as the ratio of the number of matching elements to the total number of elements in the signatures.

- **`jaccard_sim()`**: 
This function calculates the Jaccard similarity between two lists. It first calculates and returns a set containing the common elements of the two lists using the **intersection** method. Next, it calculates the **union** of the two lists as the sum of their lengths minus the length of their intersection. Finally, it calculates the Jaccard similarity as the ratio of the intersection size to the union size and returns it as a float.

In [19]:
def get_similarity(signature1, signature2):
    # Count the number of elements that are equal in both signatures
    num_matches = sum(a == b for a, b in zip(signature1, signature2))
    
    # Calculate the similarity as the ratio of matching elements to the total number of elements
    similarity = num_matches / len(signature1)
    
    # Return the calculated similarity
    return similarity

def jaccard_sim(list1, list2):
    # Calculate the intersection of the two lists by converting them to sets and using the set intersection method
    intersection = len(list(set(list1).intersection(list2)))
    
    # Calculate the union of the two lists as the sum of their lengths minus the length of their intersection
    union = (len(list1) + len(list2)) - intersection
    
    # Calculate the Jaccard similarity as the ratio of the intersection to the union
    return float(intersection) / union

To compute similarity using Min-hash signatures we used the code as seen below. 

In summary, the code performs multiple runs with different random hash functions, generates MinHash signatures for each user, calculates similarities between users, and evaluates false positives and false negatives. The average results are then calculated and printed for each run and for different numbers of hash functions tested. 

In more detail: 
- First we started by setting the maximum value for the hash functions, which in this case is R = 10009.
- Then we proceeded to define a list of numbers of hash functions to test, which included values of 50, 100, and 200.
- Additionally, we specified the similarity threshold as 0.5 and the number of runs as 5. 
- We continued by iterating over the list of hash functions, performing the following steps:
    - We created lists and variables to store the results for each run, including the lengths of similar users, the average number of similar users, and the average false positives and false negatives.
    - Within each run, we generated different random hash functions by selecting random values for 'a' and 'b'. These hash functions were used to calculate MinHash signatures for each user based on their associated movies. 
    - The similarities between pairs of users were then calculated by comparing their MinHash signatures. If the similarity was above the threshold, the pair of users was considered similar and added to the list of similar users. 
    - False positives and false negatives were also evaluated by comparing the MinHash similarity to the exact Jaccard similarity. 

The process was repeated for the specified number of runs, and the average number of similar users, false positives, and false negatives were calculated over all runs.

In [20]:
# Import library to generate random numbers
import random

# Calculate the start time of the code execution
start_time = time.time()

# Set the maximum value for the hash functions and the list of number of hash functions to test
R = 10009
num_functions_list = [50, 100, 200]

# Set the similarity threshold and the number of runs
threshold = 0.5
n_runs = 5

# Iterate over the list of number of hash functions to test
for num_functions in num_functions_list:
    # Create lists to store the results for each run
    similar_users_lengths = []
    avg_similar_users = []
    avg_false_positives = 0
    avg_false_negatives = 0

    # Print the current number of hash functions being tested
    print(f"{num_functions} Hash functions Results")
    print("-------------------------------------------")
    
    # Set the seed for reproducibility
    random.seed(1000)
    
    # Perform multiple runs with different random hash functions
    for n in range(n_runs):
        # Create a list to store the different hash functions based on the randomly generated a and b values
        hash_functions = []
        for j in range(num_functions):
            # Excluded 0 to avoid hash functions be simplified to a constant value of b.
            # Upper range was set to R-1 to avoid the hash_value be 0 for multiple pairs. 
            # eg hash_value = (10009 * 1 + 10009) % 10009 = 0
            a = random.randint(1, R-1)
            b = random.randint(0, R-1)
            hash_functions.append((a, b))
        
        # Create a dictionary to store the MinHash signatures for each user
        minhash_signatures = {}
        
        # Generate all signatures for each user
        for user_id, movies in movies_user.items():
            # Set the initial values to positive infinity to ensure that 
            # the first hash value encountered for each movie will be smaller than the initial infinity value
            signature = [float('inf')] * num_functions
            
            # Calculate hash values for each movie and store as signature the minimum one
            for movie_id in movies:
                for i, (a, b) in enumerate(hash_functions):
                    hash_value = (a * movie_id + b) % R
                    signature[i] = min(signature[i], hash_value)
                    
            # Store the MinHash signature for the user
            minhash_signatures[user_id] = signature
        
        # Create a list to store the similar users and variables to count false positives and false negatives
        similar_users = []
        false_positives = 0
        false_negatives = 0

        # Calculate the similarities between all pairs of users
        for user1 in minhash_signatures:
            signature1 = minhash_signatures[user1]
            for user2 in minhash_signatures:
                # Avoid duplicate pairs by only considering pairs where user1 < user2
                if user1 < user2:
                    signature2 = minhash_signatures[user2]
                    
                    # Calculate the similarity between the two signatures using the get_similarity function
                    similarity = get_similarity(signature1, signature2)
                    
                    # If the similarity is greater than or equal to the threshold,
                    # add the pair of users to the list of similar users
                    if similarity >= threshold:
                        similar_users.append((user1, user2))
                        
                        # Check against exact Jaccard similarity for false positives/negatives
                        jaccard_similarity = jaccard_sim(movies_user[user1], movies_user[user2])
                        if jaccard_similarity < threshold:
                            false_positives += 1
                    else:
                        # Check against exact Jaccard similarity for false positives/negatives
                        jaccard_similarity = jaccard_sim(movies_user[user1], movies_user[user2])
                        if jaccard_similarity >= threshold:
                            false_negatives += 1
        
        # Update the lists and variables with the results from this run
        similar_users_lengths.append(len(similar_users))
        avg_similar_users.append(similar_users)
        avg_false_positives += false_positives
        avg_false_negatives += false_negatives

        # Print the results for this run
        print("Number of similar users:", len(similar_users))
        print("Number of false positives:", false_positives)
        print("Number of false negatives:", false_negatives)
        print("-------------------------------------------")

    # Calculate the average number of false positives and false negatives over all runs
    avg_false_positives /= n_runs
    avg_false_negatives /= n_runs

    # Print the average results over all runs
    print("Average number of similar users:", sum(similar_users_lengths) / len(similar_users_lengths))
    print("Average number of false positives:", avg_false_positives)
    print("Average number of false negatives:", avg_false_negatives)
    print("===========================================")

# Calculate the end time and elapsed time of the code execution
end_time = time.time()
elapsed_time = end_time - start_time

# Print the elapsed time of the code execution
print(f"Elapsed time: {elapsed_time} seconds")

50 Hash functions Results
-------------------------------------------
Number of similar users: 90
Number of false positives: 81
Number of false negatives: 1
-------------------------------------------
Number of similar users: 88
Number of false positives: 80
Number of false negatives: 2
-------------------------------------------
Number of similar users: 75
Number of false positives: 66
Number of false negatives: 1
-------------------------------------------
Number of similar users: 124
Number of false positives: 116
Number of false negatives: 2
-------------------------------------------
Number of similar users: 59
Number of false positives: 53
Number of false negatives: 4
-------------------------------------------
Average number of similar users: 87.2
Average number of false positives: 79.2
Average number of false negatives: 2.0
100 Hash functions Results
-------------------------------------------
Number of similar users: 24
Number of false positives: 17
Number of false negatives: 

#### Comments

Generally, the number of hash functions used in the MinHash algorithm can affect the number of false positives and false negatives returned by the **`get_similar_users()`** function. 

Increasing the number of hash functions can improve the accuracy of the MinHash algorithm in estimating the Jaccard similarity between pairs of users. This can result in a lower number of false positives and false negatives. However, using more hash functions also increases the computational cost of the algorithm and may result in slower performance.

On the other hand, decreasing the number of hash functions can reduce the computational cost of the MinHash algorithm and make it faster. However, using fewer hash functions can also reduce the accuracy of the algorithm in estimating the Jaccard similarity between pairs of users. This can result in a higher number of false positives and false negatives.

In this project, and since the exact Jaccard similarity was already calculated, we validated the similarities from the MinHash algorithm ouput by comparing them versus the "ground truth". As it can be seen from the results, in every run and regardeless of the number of functions, the actual number of similar users was always 10 after deducting the number of false positives and by adding the number of false negatives (10 was also the actual number of similar users based on the the Jaccard similarity).

### <hr>`Task 4 - Locate similar users using LSH index`

Using a set of 200 hash functions break up the signatures into **b bands with r hash functions per band** (br=200) and implement **Locality Sensitive Hashing**.
Recall that with LSH we **first** locate users that are similar (have the same mini-signatures) across **at least one** band and **then** assess their true similarity using their **initial** representations. Use the following two instances of LSH:
- LSH instance 1: b = 25, r = 8
- LSH instance 2: b = 40, r = 5
Using each instance find the pair of users with similarity **at least 0.5** and report:
- The number of true pairs returned (true positives).
- The number of similarity evaluations performed using the initial representations.
Report the averages for **5 different runs** using different functions.
Based on the reported results, what do we **gain/loose** by using LSH instead of directly comparing users on their true representations?<hr>

#### The Process of Locating Similar Users Using the LSH Index

To begin with, we set the number of bands (**b**) and number of hash functions per band (**r**) for each instance of LSH. Specifically, we assigned 25 bands and 8 hash functions for the first instance (**b1 = 25**, **r1 = 8**), and 40 bands and 5 hash functions for the second instance (**b2 = 40**, **r2 = 5**).

Then, we created two dictionaries, **bands1** and **bands2**, to store the bands for each user.

Next, we iterated over the dataset and created the bands for each user. For each user, we split their minhash signature into bands based on the specified number of hash functions for each LSH instance. These bands were then assigned to the corresponding user in the bands1 and bands2 dictionaries.

Moving forward, we aimed to find similar users using each instance of LSH. To achieve this, we initialized two lists, **true_pairs1** and **true_pairs2**, to store the pairs of users that are identified as similar based on their LSH bands.

We employed nested loops to compare each pair of users, ensuring that each pair was considered only once. For each pair, we checked if they had at least one band in common by iterating through their bands. If a common band was found, we proceeded to calculate their Jaccard similarity using their initial movie representations.

During the similarity calculation, we kept track of the number of similarity evaluations performed. For each LSH instance, we incremented the respective **evaluations1** or **evaluations2** counter.

If the Jaccard similarity between a pair of users exceeded a specified threshold, we considered them a true pair and appended them to the corresponding true pairs list (**true_pairs1** or **true_pairs2**).

Finally, we printed the results for each instance of LSH, including the number of true pairs found and the number of similarity evaluations performed.

In [21]:
# Calculate the start time of the code execution
start_time = time.time()

import random

# Set the number of bands and hash functions for each instance of LSH
b1 = 25
r1 = 8

b2 = 40
r2 = 5

# Set the number of runs and the number of hash functions to use
n_runs = 5
num_functions = 200

# Set the maximum value for the hash functions
R = 10009

# Create lists to store the results of each run
results1 = []
results2 = []

 
# Set the seed for reproducibility
random.seed(1000)

# Perform multiple runs with different random hash functions
for n in range(n_runs):
    # Create a list to store the different hash functions based on the randomly generated a and b values
    hash_functions = []
    for j in range(num_functions):
        # Excluded 0 to avoid hash functions be simplified to a constant value of b.
        # Upper range was set to R-1 to avoid the hash_value be 0 for multiple pairs. 
        # eg hash_value = (10009 * 1 + 10009) % 10009 = 0
        a = random.randint(1, R-1)
        b = random.randint(0, R-1)
        hash_functions.append((a, b))
    
    # Create a dictionary to store the MinHash signatures for each user
    minhash_signatures = {}
    
    # Generate all signatures for each user
    for user_id, movies in movies_user.items():
        # Set the initial values to positive infinity to ensure that 
        # the first hash value encountered for each movie will be smaller than the initial infinity value
        signature = [float('inf')] * num_functions
        
        # Calculate hash values for each movie and store as signature the minimum one
        for movie_id in movies:
            for i, (a, b) in enumerate(hash_functions):
                hash_value = (a * movie_id + b) % R
                signature[i] = min(signature[i], hash_value)
                
        # Store the MinHash signature for the user
        minhash_signatures[user_id] = signature

    # Create dictionaries to store the bands for each user
    bands1 = {}
    bands2 = {}

    # Iterate over the dataset and create the bands for each user using each set of minhash signatures
    for user_id, signature in minhash_signatures.items():
        # Split the signature into bands for each instance of LSH
        bands1[user_id] = [signature[i*r1:(i+1)*r1] for i in range(b1)]
        bands2[user_id] = [signature[i*r2:(i+1)*r2] for i in range(b2)]

    # Find similar users using each instance of LSH
    true_pairs1 = []
    evaluations1 = 0

    true_pairs2 = []
    evaluations2 = 0

    for user1, bands in bands1.items():
        for user2, other_bands in bands1.items():
            if user1 < user2:
                # Check if the users have at least one band in common
                common_band = False
                for band, other_band in zip(bands, other_bands):
                    if band == other_band:
                        common_band = True
                        break
                
                # If the users have at least one band in common,
                # calculate their Jaccard similarity using their initial representations
                if common_band:
                    evaluations1 += 1
                    
                    jaccard_similarity = jaccard_sim(movies_user[user1], movies_user[user2])
                    if jaccard_similarity >= threshold:
                        true_pairs1.append((user1, user2))

    for user1, bands in bands2.items():
        for user2, other_bands in bands2.items():
            if user1 < user2:
                # Check if the users have at least one band in common
                common_band = False
                for band, other_band in zip(bands, other_bands):
                    if band == other_band:
                        common_band = True
                        break
                
                # If the users have at least one band in common,
                # calculate their Jaccard similarity using their initial representations
                if common_band:
                    evaluations2 += 1
                    
                    jaccard_similarity = jaccard_sim(movies_user[user1], movies_user[user2])
                    if jaccard_similarity >= threshold:
                        true_pairs2.append((user1, user2))

    # Store the results of this run
    results1.append((len(true_pairs1), evaluations1))
    results2.append((len(true_pairs2), evaluations2))
    
    # Print the average results for each instance of LSH
    print(f"LSH instance 1: True pairs: {len(true_pairs1)}, Similarity evaluations: {evaluations1}")
    print(f"LSH instance 2: True pairs: {len(true_pairs2)}, Similarity evaluations: {evaluations2}")
    print("-------------------------------------------------------------------------------------")


# Calculate the average number of true pairs and similarity evaluations for each instance of LSH
avg_true_pairs1 = sum([x[0] for x in results1]) / len(results1)
avg_evaluations1 = sum([x[1] for x in results1]) / len(results1)

avg_true_pairs2 = sum([x[0] for x in results2]) / len(results2)
avg_evaluations2 = sum([x[1] for x in results2]) / len(results2)

# Print the average results for each instance of LSH
print("===========================================================================================")
print(f"LSH instance 1: Average true pairs: {avg_true_pairs1}, Average similarity evaluations: {avg_evaluations1}")
print(f"LSH instance 2: Average true pairs: {avg_true_pairs2}, Average similarity evaluations: {avg_evaluations2}")

# Calculate the end time and elapsed time of the code execution
end_time = time.time()
elapsed_time = end_time - start_time

print("-------------------------------------------------------------------------------------")
# Print the elapsed time of the code execution
print(f"Elapsed time: {elapsed_time} seconds")

LSH instance 1: True pairs: 2, Similarity evaluations: 38
LSH instance 2: True pairs: 10, Similarity evaluations: 1691
-------------------------------------------------------------------------------------
LSH instance 1: True pairs: 2, Similarity evaluations: 35
LSH instance 2: True pairs: 8, Similarity evaluations: 2050
-------------------------------------------------------------------------------------
LSH instance 1: True pairs: 3, Similarity evaluations: 48
LSH instance 2: True pairs: 7, Similarity evaluations: 2447
-------------------------------------------------------------------------------------
LSH instance 1: True pairs: 3, Similarity evaluations: 53
LSH instance 2: True pairs: 8, Similarity evaluations: 2043
-------------------------------------------------------------------------------------
LSH instance 1: True pairs: 2, Similarity evaluations: 38
LSH instance 2: True pairs: 6, Similarity evaluations: 1859
-----------------------------------------------------------------

#### Comments
The code runs LSH with two different instances: b=25, r=8 and b=40, r=5. For each instance, it computes the average number of true positives and the average number of similarity evaluations over 5 different runs.

By using LSH instead of directly comparing users on their true representations, we can reduce the number of similarity evaluations that need to be performed. This can make the algorithm faster for large datasets. However, LSH is an approximate algorithm and may not find all pairs of similar users. This can result in a lower number of true positives compared to directly comparing users on their true representations.

In this project, it is noticeable that the elapsed computational time has been decreased due to the reduced number of similarity evaluations, but as it was expected the number of true pairs was decreased as well. The "ground truth" was 10 pairs but in both LSH instances we found less than 10 pairs (2.4 and 7.8 respectivelly). 